# 0 Imports

In [63]:
import pickle
import datetime

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot

from IPython.display        import Image
from IPython.core.display   import HTML

In [64]:
# Supressão da notação científica.
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.6f}'.format)

## 0.1 Funções Suporte

In [65]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [12,6]
    plt.rcParams['font.size'] = 20
    
    display( HTML('<style>.container { width: 100% !important;} </style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False)
    
    sns.set()
    
jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [66]:
def visualizacao_dados_categoricos(df):
    for i in df:
        print(f'Atributo: {i}') 
        print(f'Valores Únicos: {len(df[i].sort_values().unique())}')
        print(f'Valores Descritos: {df[i].sort_values().unique().tolist()}\n')

## 0.2 Load Data

In [67]:
with open("../data/interim/db_ajustado.pkl","rb") as arquivo:
    df = pickle.load(arquivo)
df.sample()

,subscription_name,subscription_guid,date,resource_guid,service_name,service_type,service_region,service_resource,quantity,cost,maquina,year,month,day,week_of_year,year_week,year_month
46947,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2022-05-05,43034668-a244-4031-bb4d-be5bcce22c5f,Bandwidth,Inter-Region,Intercontinental,Inter Continent Data Transfer Out - LATAM To Any,0.000362,0.000048,unicin,2022,5,5,18,2022-18,2022-05


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56933 entries, 0 to 56932
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   subscription_name  56933 non-null  object        
 1   subscription_guid  56933 non-null  object        
 2   date               56933 non-null  datetime64[ns]
 3   resource_guid      56933 non-null  object        
 4   service_name       56933 non-null  object        
 5   service_type       56933 non-null  object        
 6   service_region     56933 non-null  object        
 7   service_resource   56933 non-null  object        
 8   quantity           56933 non-null  float64       
 9   cost               56933 non-null  float64       
 10  maquina            56933 non-null  object        
 11  year               56933 non-null  int32         
 12  month              56933 non-null  int32         
 13  day                56933 non-null  int32         
 14  week_o

## 0.3 Planejamento para criação de novas Features

1. É Semana
2. É fim de semana
3. Custo Diário
4. Média Móvel Semanal
5. Média Móvel Mensal

# 3.0 F.E.

In [69]:
df.columns

Index(['subscription_name', 'subscription_guid', 'date', 'resource_guid',
       'service_name', 'service_type', 'service_region', 'service_resource',
       'quantity', 'cost', 'maquina', 'year', 'month', 'day', 'week_of_year',
       'year_week', 'year_month'],
      dtype='object')

In [ ]:
colunas = ['service_name', 'service_type', 'service_region', 'service_resource','maquina']
colunas_categoricas = df.loc[:,colunas]

In [ ]:
visualizacao_dados_categoricos(colunas_categoricas)
#del colunas_categoricas

Atributo: service_name
Valores Únicos: 15
Valores Descritos: ['Automation', 'Azure App Service', 'Azure DNS', 'Azure Monitor', 'Backup', 'Bandwidth', 'Functions', 'Key Vault', 'Log Analytics', 'Logic Apps', 'Network Watcher', 'Storage', 'Virtual Machines', 'Virtual Machines Licenses', 'Virtual Network']

Atributo: service_type
Valores Únicos: 24
Valores Descritos: ['All', 'BS Series', 'BS Series Windows', 'Basv2 Series', 'Dv3/DSv3 Series', 'Files', 'Files v2', 'Free Plan', 'General Block Blob', 'IP Addresses', 'Inter-Region', 'Premium', 'Premium Page Blob', 'Premium SSD Managed Disks', 'Private Link', 'Process', 'Queues v2', 'SQL Server Azure Hybrid Benefit', 'SQL Server Developer Edition', 'Standard HDD Managed Disks', 'Standard Page Blob', 'Standard SSD Managed Disks', 'Tables', 'Tiered Block Blob']

Atributo: service_region
Valores Únicos: 7
Valores Descritos: ['All', 'BR South', 'Global', 'Intercontinental', 'North America', 'South America', 'US East']

Atributo: service_resource
V

In [70]:
df2 = df.copy()

In [71]:
#del df

In [72]:
df2 = df2.sort_values(by=['maquina', 'date']).reset_index() #Garantir a sequencia ordenada de máquina e data
df2.drop(columns=['index'], inplace=True)

In [73]:
df2.iloc[len(df2):len(df2)-5:-1]

,subscription_name,subscription_guid,date,resource_guid,service_name,service_type,service_region,service_resource,quantity,cost,maquina,year,month,day,week_of_year,year_week,year_month
56932,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2024-10-11,40551b4c-e8be-48ed-b70b-f8d25c7de724,Storage,General Block Blob,All,Read Operations,0.001000,0.000000,unicin,2024,10,11,41,2024-41,2024-10
56931,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2024-10-11,8dbc6497-3d3a-47d6-9de4-dd42072fad66,Storage,Standard Page Blob,BR South,LRS Data Stored,0.000988,0.000114,unicin,2024,10,11,41,2024-41,2024-10
56930,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2024-10-11,2dba9884-2518-4427-90ff-d2c100c5f775,Storage,Standard SSD Managed Disks,BR South,E6 LRS Disk,0.025536,0.228798,unicin,2024,10,11,41,2024-41,2024-10
56929,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2024-10-11,2f2a1523-d8af-5cea-9b91-d0285e17e343,Backup,All,BR South,Azure VM Protected Instances,0.012096,0.120960,unicin,2024,10,11,41,2024-41,2024-10


In [74]:
df2['dia_da_semana'] = df2['date'].dt.dayofweek.astype(int)

In [75]:
df2['fim_de_semana'] = df2['dia_da_semana'].apply(lambda x : 1 if x >= 5 else 0)

In [76]:
df2['custo_diario'] = df2.groupby(['maquina','date'])['cost'].transform('sum')

In [77]:
df2.sample(5)

,subscription_name,subscription_guid,date,resource_guid,service_name,service_type,service_region,service_resource,quantity,cost,maquina,year,month,day,week_of_year,year_week,year_month,dia_da_semana,fim_de_semana,custo_diario
26592,Microsoft Azure Sponsorship,0b1856f5-ffb1-4fb2-9b31-ebc3ddeacf68,2024-09-23,37a224be-47e1-42bd-b2aa-f08060e965ee,Storage,Tables,BR South,LRS Data Stored,0.000144,0.000000,ceaec,2024,9,23,39,2024-39,2024-09,0,0,7.070502
51504,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2023-12-30,8dbc6497-3d3a-47d6-9de4-dd42072fad66,Storage,Standard Page Blob,BR South,LRS Data Stored,0.001248,0.000144,unicin,2023,12,30,52,2023-52,2023-12,5,1,5.272497
13345,Microsoft Azure Sponsorship,0b1856f5-ffb1-4fb2-9b31-ebc3ddeacf68,2023-03-05,ce957933-8287-4f8b-83ee-53a5cbe056d9,Storage,Standard SSD Managed Disks,All,E4 LRS Disk Operations,44.680400,0.089358,ceaec,2023,3,5,9,2023-09,2023-03,6,1,9.144229
44894,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2022-11-28,37a224be-47e1-42bd-b2aa-f08060e965ee,Storage,Tables,BR South,LRS Data Stored,0.000176,0.000000,unicin,2022,11,28,48,2022-48,2022-11,0,0,9.057281
39861,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2022-06-17,b9e5e77c-a0b3-4a2c-9b8b-57fa54f31c52,Storage,Tables,All,Batch Write Operations,0.029200,0.000000,unicin,2022,6,17,24,2022-24,2022-06,4,0,8.596325


## 3.1 Próximas implementações

In [ ]:
#df_fe = df2[['maquina','date','cost']].copy()

In [60]:
df_fe['custo_media_movel_semanal'] = df_fe.loc[:,['maquina','date','cost']].groupby(
                                    ['maquina']
                                    )['cost'].transform(
                                                lambda x: x.rolling(
                                                    window=7,
                                                    min_periods=1
                                                    ).sum()
                                                )

In [61]:
df_fe['custo_media_movel_semanal_teste'] = df_fe.groupby(
                                                    ['maquina']
                                                    )['cost'].transform(
                                                                lambda x: x.rolling(
                                                                    window=7,
                                                                    min_periods=1
                                                                    ).sum()
                                                                ).reset_index(
                                                                    level=0,
                                                                    drop=True
                                                                    )

In [62]:
df_fe['custo_media_movel_semanal_data'] = df_fe.groupby(
                                            ['maquina',
                                            'date'])['cost'].transform(
                                                        lambda x: x.rolling(
                                                            window=7,
                                                            min_periods=1
                                                            ).sum()
                                                        )

In [40]:
df_fe['custo_media_movel_mensal'] = df_fe.groupby(
                                    ['maquina']
                                    )['cost'].transform(
                                                lambda x: x.rolling(
                                                    window=30,
                                                    min_periods=1
                                                    ).sum()
                                                )

In [49]:
aux2 = df_fe.loc[(df_fe['date']>='2024-09-12') & (df_fe['maquina'] == 'ceaec'),:].groupby(['maquina','date'])['cost'].sum().reset_index()

In [50]:
aux2['cost'].sum()

208.966287

In [78]:
#PRECISO FAZER UM MERGE AQUI ENTRE df_fe E df2
df_fe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56933 entries, 0 to 56932
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   maquina                          56933 non-null  object        
 1   date                             56933 non-null  datetime64[ns]
 2   cost                             56933 non-null  float64       
 3   custo_media_movel_semanal        56933 non-null  float64       
 4   custo_media_movel_semanal_teste  56933 non-null  float64       
 5   custo_media_movel_semanal_data   56933 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 2.6+ MB


In [52]:
aux= df2.loc[(df2['maquina']=='unicin') & (df2['date']=='2024-10-11'),
        ['date',
        'maquina',
        'cost',
        'custo_diario',
        'custo_media_movel_semanal',
        'custo_media_movel_semanal_teste']]
aux

,date,maquina,cost,custo_diario,custo_media_movel_semanal,custo_media_movel_semanal_teste
32135,2024-10-11,unicin,0.080598,4.242862,0.915558,0.163302
33077,2024-10-11,unicin,0.000000,4.242862,0.675558,1.266491
35542,2024-10-11,unicin,0.007657,4.242862,0.538687,0.163251
36545,2024-10-11,unicin,0.267403,4.242862,0.644810,2.152373
37548,2024-10-11,unicin,0.148602,4.242862,0.504404,2.219016
38123,2024-10-11,unicin,0.068400,4.242862,0.572660,2.330395
39117,2024-10-11,unicin,0.000000,4.242862,0.572660,0.182758
40024,2024-10-11,unicin,1.089000,4.242862,1.581062,4.878868
41503,2024-10-11,unicin,0.073764,4.242862,1.654826,2.253994
44027,2024-10-11,unicin,0.000000,4.242862,1.647169,0.002252


In [21]:
aux['cost'].sum()

5.485406

In [41]:
aux_df = df2.loc[(df2['maquina']=='unicin') & (df2['date']>='2024-09-12'),
        ['date',
        'maquina',
        'cost',
        'custo_media_movel_semanal',
        'custo_media_movel_mensal']
        ].groupby(['maquina','date']).sum().sort_values(by='date',ascending=False).reset_index()
aux_df

,maquina,date,cost,custo_media_movel_semanal,custo_media_movel_mensal
0,unicin,2024-10-11,4.242862,30.172204,5.153284
1,unicin,2024-10-10,5.629300,39.405100,5.634637
2,unicin,2024-10-09,5.640620,39.484340,5.622057
3,unicin,2024-10-08,5.600468,39.203276,5.490353
4,unicin,2024-10-07,5.365676,37.559732,5.341884
5,unicin,2024-10-06,5.314549,37.201843,5.325528
6,unicin,2024-10-05,5.361065,37.527455,5.291303
7,unicin,2024-10-04,5.238780,36.671460,5.282548
8,unicin,2024-10-03,5.283107,36.981749,8.022691
9,unicin,2024-10-02,11.152205,78.065435,8.432701


In [38]:
aux_df.loc[:,'cost'].sum()

167.62592

In [39]:
aux_df.loc[aux_df['date']>='2024-10-05',['cost']].sum()

cost   37.154540
dtype: float64

# 3.9 Export PKL

In [ ]:
with open("../data/interim/3_0_feature_engineering.pkl","wb") as arquivo:
    pickle.dump(df2, arquivo)